<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Гибридные изображения. Эйнштейн-Монро

### Examples - Astrophysics
<section class="post-meta">
By Jonas Tjemsland, Andreas Krogen, Håkon Ånes and Jon Andreas Støvneng
</section>
Last edited: March 22nd 2018 
___

## Вступление

Глядя на изображение ниже, кого вы видите? Знаменитого физика Альберта Эйнштейна или популярный секс-символ 1950-х годов Мэрилин Монро?

С небольшого расстояния можно увидеть четкое изображение Эйнштейна, и только размытое искажение намекает на наличие наложенного изображения. Но при взгляде издалека или прищурившись, мельчайшие детали Эйнштейна размываются, и появляется  лицо Монро! Обратите также внимание, что если зритель близорук, он может видеть Монро на нормальном расстоянии обзора.

<img src="images/einstein-monroe.png" width="300">

**Рисунок 1:** За этим великим человеком стоит красивая женщина [4].

Изображение является одним из самых известных примеров так называемого "гибридного изображения". Оно создается путем объединения двух изображений, одно из которых фильтруется фильтром низких частот, а другое - фильтром высоких частот [2]. Эта концепция основана на том, как мы, люди, обрабатываем визуальный ввод. Схема создания изображений с помощью этой оптической иллюзии была разработана Од Оливой из Массачусетского технологического института и Филиппом Г. Шинсом из Университета Глазго.

Одним из физических объяснений этого эффекта является пространственное разрешение наших глаз, оцениваемое по критерию Рэлея, $\Delta l_{min} = 1,22\lambda R/D$ [1]. Здесь $D$ - диаметр апертуры объектива, $\lambda$ - длина волны света, а $R$ - расстояние до объекта или изображения. В качестве примера пусть $R=25\mathrm{cm}$ (идеальное расстояние считывания), $D\approx4\mathrm{mm}$ (диаметр зрачков в наших глазах) и $\lambda=550\mathrm{nm}$ (середина видимого спектра). Затем самые мелкие детали, которые мы можем разрешить на изображении, - это $\Delta l_{min} \approx$ 0.04 мм. Худшее разрешение, т.e. увеличение $\Delta l_{min}$ достигается увеличением расстояния до изображения $R$ или прищуриванием и, таким образом, уменьшением $D$. Следовательно, более тонкие детали Эйнштейна не улавливаются, когда $\Delta l_{min}$ увеличивается, и образ Монро имеет приоритет.

Используя инструменты, представленные в нашей записной книжке *[Фильтрация изображений с использованием Фурье Transform](https://nbviewer.jupyter.org/url/www.numfys.net/media/notebooks/image_filtering_using_fourier_transform.ipynb)*, должно позволить легко проанализировать концепцию гибридных изображений. В этом блокноте мы пытаемся использовать фильтр низких и высоких частот для извлечения экстрактов Монро и Эйнштейна соответственно. Ближе к концу мы создаем свой собственный, упрощенный гибридный образ!

Мы начинаем с импорта пакетов, установки параметров рисунка и определения функций, определенных в вышеупомянутом блокноте по фильтрации изображений.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.fftpack as fft
from skimage import io # used to read images from url
import warnings
warnings.filterwarnings("ignore")

def FFT(image):
    """ Compute the 2-dimensional discrete Fourier Transform of an image
    and shift the zero-frequency component to the center of the spectrum.
    
    Parameters:
        image: array-like, shape (m, n), can be complex. Image being transformed.
    Returns: complex ndarray, shape (m, n). Frequency spectrum.
    """
    return np.fft.fftshift(np.fft.fft2(image))

def IFFT(spectrum):
    """ Shift the zero-frequency components to the edges of the spectrum and 
    compute the inverse 2-dimensional discrete Fourier Transform of an image.
    
    Parameters:
        image: array-like, shape (m, n), can be complex. Spectrum being transformed.
    Returns: complex ndarray, shape (m, n). Spacial domain.
    """
    return np.fft.ifft2(np.fft.fftshift(spectrum))

def filter_spectrum(spectrum, filter_type=None, val=50):
    """ Filter the spectrum of an image using a gaussian high pass or low pass filter.
    
    Parameters:
        spectrum:    array-like, shape (m, n), can be complex. Spectrum being filtered.
        filter_type: string. Filter name, {'gaussian_highpass', 'gaussian_lowpass'}.
        val:         float. Filter parameter.
    Returns:         complex 2d-array, shape (m,n). Filtered spectrum.
    """
    m, n = np.shape(spectrum)
    x, y = np.meshgrid(np.arange(m), np.arange(n))
    R2 = np.transpose((x - m/2)**2 + (y - n/2)**2)
    if (filter_type == 'gaussian_highpass'):
        return spectrum*(1 - np.exp(-val*R2))
    elif (filter_type == 'gaussian_lowpass'):
        return spectrum*np.exp(-val*R2)
    elif (filter_type != None):
        raise ValueError('%s is not a valid filter!' % filter_type)
    return spectrum

def normalise(image):
    """ Normalise an image such that the minimal pixel value becomes 0 and the maximum 
    pixel value becomes 1. Only the real value of the image is used in case of 
    numerical rounding errors during the Fourier Transforms.
    
    Parameters:
        image: array-like, shape (m, n), can be complex. Image being transformed.
    Returns: real 2d-array(). Normalised image.
    """
    image = np.real(image)
    return (image - np.min(image))/(np.max(image) - np.min(image))

Давайте импортируем изображение Эйнштейна-Монро, выполним преобразование Фурье, чтобы получить частотный спектр, а затем визуализируем результат.

In [ ]:
# Import image, originally downloaded from [3]
img = io.imread('https://www.numfys.net/media/notebooks/images/einstein-monroe.png',
                as_grey=True)

# Compute the frequency spectrum
spec = FFT(img)

# Plot the result
plt.subplot(121)
plt.imshow(img, cmap='gray')
plt.subplot(122)
plt.imshow(np.log(abs(spec)), cmap='gray');

Теперь все готово для фильтрации изображения! Мы хотим отфильтровать так, чтобы был виден только один из двух человек. Имейте в виду, что полученные изображения будут иметь более низкое качество, чем исходные изображения (почему?).

## Извлечение частот. Высокие = Эйнштейн, низкие = Монро

Теперь мы используем фильтр высоких частот Гаусса, чтобы исключить низкие частоты, чтобы получить Эйнштейна, и фильтр низких частот Гаусса, чтобы исключить высокие частоты, чтобы получить Монро. После попыток с различными исключающими значениями мы получаем изображения, на которых Эйнштейна и Монро можно увидеть отдельно.

In [ ]:
# Filter spectra
filtered_spec_einstein = filter_spectrum(spec, filter_type='gaussian_highpass', val=0.001)
filtered_spec_monroe = filter_spectrum(spec, filter_type='gaussian_lowpass', val=0.01)

# Transform into filtered images
filtered_img_einstein = np.real(IFFT(filtered_spec_einstein))
filtered_img_monroe = np.real(IFFT(filtered_spec_monroe))

# Plot result
plt.subplot(121)
plt.imshow(filtered_img_einstein, cmap='gray');
plt.subplot(122)
plt.imshow(filtered_img_monroe, cmap='gray');

## Создайте свой собственный гибридный образ

Пришло время *создать* гибридный образ! Мы будем использовать образ Леонарда Нимоя в роли Спока [5] и Уильяма Шатнера в роли Джеймса Т. Кирка [6] из "Звездного пути". Обратите внимание, что изображения должны быть правильно выровнены для достижения наилучшего эффекта оптической иллюзии. Некоторые дополнительные советы описаны в статье [гибридные изображения]( http://cvcl.mit.edu/hybrid/OlivaTorralb_Hybrid_Siggraph06.pdf) Од Олива, Антонио Торральба и Филипп Г. Шинс с 2008 года. Мы начинаем с импорта изображений и их визуализации.

In [ ]:
spock = io.imread('https://www.numfys.net/media/notebooks/images/spock.jpg', True)
kirk = io.imread('https://www.numfys.net/media/notebooks/images/kirk.jpg', True)
plt.subplot(121)
plt.imshow(spock, cmap='gray')
plt.subplot(122)
plt.imshow(kirk, cmap='gray');

Теперь мы фильтруем изображения. Пусть Спок будет отфильтрован на высоких частотах, а Кирк - на низких. Поиграйте с параметрами!

In [ ]:
# Filter Spock
spock_spec = FFT(spock)
filtered_spock_spec = filter_spectrum(spock_spec, filter_type='gaussian_highpass', val=0.001)
filtered_spock = np.real(IFFT(filtered_spock_spec))

# Filter Kirk
kirk_spec = FFT(kirk)
filtered_kirk_spec = filter_spectrum(kirk_spec, filter_type='gaussian_lowpass', val=0.005)
filtered_kirk = np.real(IFFT(filtered_kirk_spec))

# Plot the result
plt.subplot(121)
plt.imshow(filtered_spock, cmap='gray')
plt.subplot(122)
plt.imshow(filtered_kirk, cmap='gray');

Теперь мы нормализуем изображения, складываем их вместе и строим график результата. И у нас есть свой гибридный образ!

In [ ]:
# Normalise the individual images
filtered_spock = normalise(filtered_spock)
filtered_kirk = normalise(filtered_kirk)

# Create the hybrid image
spock_kirk_hybrid = filtered_spock + filtered_kirk

# Plot the result
plt.imshow(spock_kirk_hybrid, cmap='gray');

## Дальнейшая работа и улучшения

* Как такое проделать для цветного изображения?
* Можно ли добавить более двух изображений в гибридное изображение?
* Существуют ли какие-либо приложения для гибридных изображений (например, рекламные ролики)?

## Resources

[1] Wikipedia. *Angular resolution* https://en.wikipedia.org/wiki/Angular_resolution, 10-25-106 [acquired 11-08-2016]

[2] Aude Oliva, Antonio Torralba, Philippe. G. Schyns. *Hybrid Images*, http://cvcl.mit.edu/hybrid/OlivaTorralb_Hybrid_Siggraph06.pdf, 2008 [aquired 10-29-2016]    

[3] Wikipedia. *Hybrid image*, https://en.wikipedia.org/wiki/Hybrid_image, 10-26-2007 [aquired 10-28-2016]   

### Pictures:

[4] Gregory T. Huang: Einstein-Monroe, https://www.newscientist.com/article/mg19325971-600-hybrid-images-now-you-see-them/?DCMP=ILC-Top5&;href=images, 03-28-2007 [aquired 10-28-2016]  

[5] NBC television: Leonard Nimoy as Spock, https://commons.wikimedia.org/wiki/File:Leonard_Nimoy_as_Spock_1967.jpg, 1967 [aquired 10-28-2016]  

[6] NBC television: William shatner as James T. Kirk, https://commons.wikimedia.org/wiki/File:Star_Trek_William_Shatner.JPG, 1967 [aquired 10-28-2016]